# Projet 2 de BioInformatique
Rémy Detobel - 000408013                
[rdetobel@ulb.ac.be](mailto:rdetobel@ulb.ac.be)        

------------------------

Le but de ce mini projet est de créer une matrice de substitution spécifiquement pour des familles de protéines.  Pour ce faire, nous allons nous baser sur les informations présentes sur le site [BLOCKS](http://blocks.fhcrc.org/).  Dans ce projet nous nous concentrerons sur la famille SH3 et PDZ.              
Pour faire ces matrices de substitution, nous utiliserons l'approche BLOSUM (qui sera décrite dans ce rapport).

#### Création d'un objet Bloc        
Afin de structurer le code, nous allons créer un objet `Bloc`, celui-ci représentera les différents blocs présents dans une famille.         
            
Lors de sa création, l'objet `Bloc` va calculer plusieurs valeurs afin d'accélérer le calcul de la matrice de subsitution.       
On va donc commencer par calculer les groupes pouvant être formés pour ce bloc.   Pour se faire, on parcourt toutes les séquences et ensuite toutes les séquences présentes dans les groupes déjà formés.  Si une séquence correspond (c'est à dire que le nombre d'acides aminés correspondant est plus grand ou égal que la proportion donnée lors de la création de l'objet), on l'ajoute dans le bloc et on passe à la séquence suivante.  Si aucun groupe ne correspond à cette séquence, on créé un nouveau groupe.        
          
Une fois les groupes formés, on peut calculer la proportion avec laquelle une lettre apparait dans une colonne donnée.  C'est à dire le nombre de fois qu'elle apparait par rapport au nombre de séquences qu'il y a dans ce groupe.             
On va ensuite calculer les fréquences pondérés pour chaque association d'acides aminés.  Pour se faire, on implémente une double boucle parcourant tous les groupes présent dans ce bloc.  On va ensuite parcourir chaque séquence de gauche à droite, en colonne.  Pour chacune des lettres présente dans le premier groupe à une colonne définie, on parcourt toutes les lettres pouvant être présentes dans cette même colonne mais dans le second groupe.  De cette manière on constituera toutes les possibilités d'association.  On stocke directement ces résultats dans un dictionnaire où la clef représente l'association des deux acides aminés.

In [2]:
class Bloc:

    # Création d'un bloc à partir d'une liste de séquences
    def __init__(self, list_seq, pourcIdentite = 0.5):
        self.list_seq = list_seq
        self.allGroupe = []
        self.pourcIdentite = pourcIdentite
        

        self.calculGroupe()
        print("Nombre de groupe: " + str(len(self.allGroupe)))
        self.calculProportion()


    # Permet de calculer les groupes
    def calculGroupe(self):
        # On parcourt toutes les séquences
        for testSeq in self.list_seq:
            groupeTrouve = False
            
            # On parcourt tous les blocs
            iGroupe = 0
            while iGroupe < len(self.allGroupe) and not groupeTrouve:
                groupe = self.allGroupe[iGroupe]
                iGroupe += 1
                
                # Mais également toutes les séquences de ce groupe
                iSeq = 0
                while iSeq < len(groupe) and not groupeTrouve:
                    seq = groupe[iSeq]
                    iSeq += 1
                    # On regarde si la séquence actuelle est "identique" à la séquence de recherche
                    if(self.isSeqIdent(testSeq, seq)):
                        groupe.append(testSeq)
                        groupeTrouve = True
            
            # Si on n'a pas trouvé de groupe, on en créé un
            if(not groupeTrouve):
                self.allGroupe.append([testSeq])
    
    
    # On calcule la proportion de chaque lettre et leur combinaison dans ce bloc
    def calculProportion(self):
        # On divise le nombre d'occurence d'une lettre dans une colonne 
        # par le nombre d'élements dans le groupe
        allCacheLettre = []
        for groupe in self.allGroupe:
            cacheLettre = [{} for i in range(len(groupe[0]))]

            nbrSeq = len(groupe)
            for seq in groupe:
                for col in range(len(seq)):
                    lettre = seq[col]
                    
                    if(lettre in cacheLettre[col]):
                        cacheLettre[col][lettre] += 1/nbrSeq
                    else:
                        cacheLettre[col][lettre] = 1/nbrSeq

            allCacheLettre.append(cacheLettre)

        # On va ensuite combiner tous ces résultats pour former des paires
        self.resultDictionnary = {}

        for iGroupeUn in range(len(allCacheLettre)):
            for iGroupeDeux in range(len(allCacheLettre)):
                if(iGroupeUn != iGroupeDeux):
                    for col in range(len(allCacheLettre[iGroupeUn])):

                        for keyA, valeurA in allCacheLettre[iGroupeUn][col].items():

                            for keyB, valeurB in allCacheLettre[iGroupeDeux][col].items():
                                if(keyA != keyB or iGroupeDeux < iGroupeUn):
                                    dicKey = keyA+keyB
                                    if(dicKey in self.resultDictionnary):
                                        self.resultDictionnary[dicKey] += valeurA * valeurB
                                    else:
                                        self.resultDictionnary[dicKey] = valeurA * valeurB


    # Permet d'avoir la proportion pondérée de l'apparition d'une 
    # combinaison de lettres dans les différents groupes
    def getProportionLettre(self, lettreA, lettreB):
        key = lettreA+lettreB
        if(key in self.resultDictionnary):
            return self.resultDictionnary[key]
        else:
            return 0


    # Permet de savoir si deux séquences sont "identiques" 
    # (en fonction du paramètre donné lors de l'initialisation)
    def isSeqIdent(self, seq1, seq2):
        if(len(seq1) != len(seq2)):
            raise SyntaxError("Les séquences n'ont pas le même " \
                "nombre de caractère %d != %d " % (seq1, seq2))

        objectif = self.pourcIdentite*len(seq1)
        total = 0
        i = 0
        while i < len(seq1) and total < objectif:
            if(seq1[i] == seq2[i]):
                total += 1
            i += 1

        return total >= objectif


    # Permet d'afficher les groupes de ce bloc
    def printGroupe(self):
        i = 0
        for groupe in self.allGroupe:
            i += 1
            print("Groupe " + str(i) + ": " + " ".join(groupe))
        print("")

#### Création d'un objet Score        
C'est dans cet objet que sera calculé la matrice de substitution.  La création de l'objet `Score` se fait à partir d'un ou plusieurs objets `Bloc`.  En effet, on va créer une matrice basée sur une famille et donc potentiellement plusieurs Blocs.               
              
La création de l'objet `Score` va provoquer le calcul et la structure de plusieurs données.   Avant tout, on vérifie que la liste fournie à `Score` n'est pas vide.  On initialise ensuite une demi matrice (en effet, la matrice de substitution est symétrique).         
On va ensuite calculer la matrice des fréquences pondérées.  L'objet bloc a déjà calculé chaque valeur pour ses propres données.  Nous n'avons donc plus qu'à additionner les différentes valeurs présentes dans ces blocs.    
           
Une fois la matrice des fréquences pondérées calculée, il nous faut calculer la probabilité d'occurence de chaque case de la matrice (c'est à dire chaque association d'acide aminé).  Pour faire cela, on calcule la somme de toutes les cellules de la demi matrice.  On va ensuite diviser toutes les valeurs de la matrice par cette somme.      
                
On va ensuite calculer la fréquence prévue  par résidu.  C'est à dire:
$$p_a = q_{a,a} + \frac{1}{2} \sum\limits_{a \neq b} q_{a, b}$$       
Donc, pour chaque acide aminé, on stocke sa fréquence prévue par résidu dans une liste.  Ces informations nous permettront de calculer plus simplement la matrice de substitution.            
                
Enfin, nous créons la matrice de substitution en utilisant toutes les données calculées précédemment.  Pour calculer la valeur à mettre dans la ligne `a` et la colonne `b`, on utilisera cette forumule:
$$s_{a, b} = 2 \log_2(\frac{q_{a, b}}{e_{a, b}})$$     
La valeur `e` se calcule comme ceci:
$$
\begin{eqnarray}
e_{aa} &=& p_a^2\\
e_{ab} &=& 2p_b p_a\\
\end{eqnarray}
$$

In [3]:
from math import log2 # Import du logarithme en base 2

class Score:

    # Lettres utilisées pour représenter des acides aminés dans les matrices BLOSUM
    BLOSUM_LETTER = "ARNDCQEGHILKMFPSTWYVBZX"
    
    
    #Création de l'objet Score à partir d'une liste de blocs
    def __init__(self, listBloc, debug = False):
        self.listBloc = listBloc
        self.debug = debug

        # Si la liste des blocs est vide, impossible de calculer un score
        if(len(listBloc) == 0):
            raise ValueError("La liste des blocs ne peut pas être vide")
        
        if(self.debug):
            print("Calcul du Score avec " + str(len(listBloc)) + 
                " bloc" + ("s" if len(listBloc) > 1 else "") + "\n")

        self.initMatriceFreqPondere()
        self.calculMatriceFreqPond()
        self.calculProbaOccurence()

        self.calculListProbaResidu()
        self.calculMatriceScore()


    # Permet d'initialiser une matrice des fréquences pondérées
    def initMatriceFreqPondere(self):
        # Pour éviter de faire des calculs inutiles, on ne va utiliser qu'une demi matrice
        # En effet, celle-ci est symétrique
        self.matFreqPond = [[0 for i in range(j+1)] for j in range(len(Score.BLOSUM_LETTER))]


    # On calcule les fréquences pondérés à mettre dans la matrice
    def calculMatriceFreqPond(self):
        if(self.debug): # Si on est en mode debug, on affiche les groupes
            for bloc in self.listBloc:
                bloc.printGroupe()

        # Pour chaque ligne, chaque colonne et chaque groupe
        for ligne in range(len(self.matFreqPond)):
            for col in range(len(self.matFreqPond[ligne])):
                for bloc in self.listBloc:
                    self.matFreqPond[ligne][col] += bloc.getProportionLettre(Score.BLOSUM_LETTER[ligne], \
                                                                             Score.BLOSUM_LETTER[col])
        if(self.debug):
            self.printMatrice(self.matFreqPond)


    # Calcul des probabilités d'occurrence
    def calculProbaOccurence(self):
        sumDemiMatrice = sum([sum(elem) for elem in self.matFreqPond])

        if(self.debug):
            print("\nSomme de la demi-matrice: " + str(sumDemiMatrice))

        for ligne in range(len(self.matFreqPond)):
            for colonne in range(len(self.matFreqPond[ligne])):
                self.matFreqPond[ligne][colonne] = (self.matFreqPond[ligne][colonne] / sumDemiMatrice)
        if(self.debug):
            print("")
            self.printMatrice(self.matFreqPond, 5)


    # Pour calculer le taux de log-chance on calcule la probabilité d'occurence pour un
    # acide aminé.  Cela forme donc une liste
    def calculListProbaResidu(self):
        if(self.debug):
            strDebug = ""

        self.listFreqResidu = []
        for lettre in range(len(Score.BLOSUM_LETTER)):
            total = 0
            for index in range(len(Score.BLOSUM_LETTER)):
                if(index != lettre):
                    total += self.matFreqPond[max(index, lettre)][min(index, lettre)]

            self.listFreqResidu.append(self.matFreqPond[lettre][lettre] + (total/2))
            if(self.debug):
                strDebug += (Score.BLOSUM_LETTER[lettre] + "=" 
                        + str(round(self.listFreqResidu[-1], 2)) + " ")

        if(self.debug):
            print("\nListe de la probabilité d'occurence d'un alignement:" + \
                  strDebug + "\n")


    # Permet de calculer la matrice score avec toutes les informations faites précédemment
    def calculMatriceScore(self):
        self.resMatrice = [] # resMatric contiendra la matrice finale
        for ligne in range(len(Score.BLOSUM_LETTER)):
            self.resMatrice.append([])
            for colonne in range(ligne+1):
                currentMatrice = self.matFreqPond[ligne][colonne]

                if(currentMatrice != 0):
                    if(ligne == colonne):
                        valeur = (self.listFreqResidu[ligne]**2)
                    else:
                        valeur = (2 * self.listFreqResidu[ligne] * self.listFreqResidu[colonne])

                    self.resMatrice[ligne].append(2*log2(currentMatrice / valeur))
                else:
                    self.resMatrice[ligne].append(0)


    # Permet d'afficher une matrice
    def printMatrice(self, matrice, max_nbr = 4):
        for i in range(len(matrice)):
            # Ligne avec les indices
            if(i == 0):
                print("  ", end="")
                for j in range(len(Score.BLOSUM_LETTER)):
                    print(Score.BLOSUM_LETTER[j].center(max_nbr, ' '), end="|")
                print()
            
            # Affichage des lignes
            for j in range(len(matrice[i])):
                if(j == 0):
                    # Affiche l'indice
                    print(Score.BLOSUM_LETTER[i], end="|")

                print(str(round(matrice[i][j], max(max_nbr-3, 1))).center(max_nbr, ' '), end="|")
            print() # Retour à la ligne
            
    
    # Permet d'afficher le résultat (la matrice de substitution)
    def printResMatrice(self):
        self.printMatrice(self.resMatrice)

#### Chargement de fichiers de tests               
L'algorithme est maintenant prêt.  Il ne reste plus qu'à importer les différents blocs d'une famille.  On définit donc une fonction `importBloc` qui s'occupera de lire le fichier fourni en paramètre, de le parser et de créer les objets utiles à la création de la matrice de substitution.

In [4]:
def importBloc(nom_fichier, proportion = 0.5, debug = False):

    # Ouverture du fichier
    fichier = open(nom_fichier, 'r')

    listBloc = []

    i = 0
    seq = 0
    for ligne in fichier:
        ligne = ligne.strip()

        if(ligne[0] == '>'):
            i = 0
            seq += 1

        else:
            if(seq == 1):
                listBloc.append([ligne])
            else:
                if(len(listBloc) > i):
                    listBloc[i].append(ligne)
                else:
                    raise SyntaxError("Erreur de taille dans la" \
                        " liste: %d | i: %d" % (len(listBloc), i))
            i += 1

    return Score([Bloc(seq, proportion) for seq in listBloc], debug)
    

##### Test 1
Pour ce premier test, nous allons utiliser les mêmes valeurs que celle présentées en cours.      
L'algorithme va donc commencer par calculer 3 groupes différents.   On peut voir juste après ces groupes la matrice de fréquences pondérées.           
On additionne ensuite toutes les valeurs présentes dans cette matrice.  On divise toutes les valeurs de la matrice par cette somme (en l'occurence, dans le cas présent 15).        
Après cela, on calcule la probabilité d'occurence d'un alignement de résidu a et b.  Algébriquement, cela se tratuit par l'addition de la valeur se trouvant sur la diagonale avec la moitié de la somme de toutes les valeurs sur la même colonne.            
Il ne reste plus qu'à calculer le taux de log-chance comme expliqué ci-dessus.

In [5]:
##### TEST 1 #####
testBloc = importBloc("TEST.fasta", 0.5, True)
testBloc.printResMatrice()

Nombre de groupe: 3
Calcul du Score avec 1 bloc

Groupe 1: ATCKQ ATCRN ASCKN SSCRN
Groupe 2: SDCEQ SECEN
Groupe 3: TECRQ

   A  | R  | N  | D  | C  | Q  | E  | G  | H  | I  | L  | K  | M  | F  | P  | S  | T  | W  | Y  | V  | B  | Z  | X  |
A| 0  |
R| 0  |0.5 |
N| 0  | 0  |0.4 |
D| 0  | 0  | 0  | 0  |
C| 0  | 0  | 0  | 0  |3.0 |
Q| 0  | 0  |1.8 | 0  | 0  |0.9 |
E| 0  |1.5 | 0  |0.5 | 0  | 0  |0.5 |
G| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
H| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
I| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
L| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
K| 0  |0.5 | 0  | 0  | 0  | 0  |0.5 | 0  | 0  | 0  | 0  | 0  |
M| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
F| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
P| 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |
S|0.8 | 0  | 0  |0.2 | 0  | 0  |0.8 | 0  | 0  | 0  | 0  | 0  | 0  | 0  | 0  |0.2 |
T|0.8 | 0  | 0  |0.2 | 0  | 0  |

### Création des matrices SH3 et PDZ
Nous alons maintenant créer 2 matrices par famille.  La première aura 40% d'identité entre les séquences faisant partie d'un même groupe et la seconde 70%.

In [6]:
##### SH3 40 #####
print("SH3 40%")
sh3_40 = importBloc("SH3.fasta", 0.4)
sh3_40.printResMatrice()

##### SH3 70 #####
print("\nSH3 70%")
sh3_70 = importBloc("SH3.fasta", 0.7)
sh3_70.printResMatrice()

##### PDZ 40 #####
print("\nPDZ 40%")
pdz_40 = importBloc("PDZ.fasta", 0.4)
pdz_40.printResMatrice()

##### PDZ 70 #####
print("\nPDZ 70%")
pdz_70 = importBloc("PDZ.fasta", 0.7)
pdz_70.printResMatrice()

SH3 40%
Nombre de groupe: 28
Nombre de groupe: 22
Nombre de groupe: 6
Nombre de groupe: 10
   A  | R  | N  | D  | C  | Q  | E  | G  | H  | I  | L  | K  | M  | F  | P  | S  | T  | W  | Y  | V  | B  | Z  | X  |
A|3.0 |
R|-1.6|1.4 |
N|-1.3|-0.0|1.8 |
D|-1.2|0.2 |2.6 |3.2 |
C|3.3 |-3.0|-2.2|-4.6|3.2 |
Q|-0.2|0.8 |0.4 |1.2 |-3.0|0.1 |
E|-1.3|0.7 |0.8 |2.3 |-2.4|1.3 |2.4 |
G|1.0 |-0.6|2.4 |1.4 |-0.2|0.9 |-0.8|4.0 |
H|-1.4|0.8 |-0.1|-1.3|-0.9|0.2 |-0.3|-3.8|2.9 |
I|-2.4|-0.9|-2.7|-2.8|-2.5|-1.2|-1.4|-3.0|-1.9|2.3 |
L|-2.0|-0.6|-2.0|-2.7|-1.4|-1.4|-1.3|-2.2|-1.4|2.7 |2.2 |
K|-0.7|1.2 |-0.3|-0.4|-2.2|1.0 |0.9 |-1.4|0.5 |-1.2|-1.2|1.5 |
M|-0.9|-2.2|-3.4|-1.7|0.2 |-1.0|-0.3|-4.5|-2.0|1.9 |1.8 |-0.2|1.4 |
F|-2.1|-1.2|-0.0|-3.6|2.1 |-2.2|-2.9|-4.5|0.4 |0.3 |0.3 |-1.1|1.1 |3.6 |
P|0.9 |-1.1|-0.2|0.0 |-0.8|0.5 |-1.0|1.2 |-4.0|-3.2|-2.5|0.6 |-1.2|-2.1|4.9 |
S|-0.3|0.9 |1.2 |1.1 |-2.5|0.3 |0.6 |0.4 |1.2 |-3.4|-2.4|0.7 |-1.2|-2.6|0.8 |2.1 |
T|1.3 |0.4 |-1.2|-1.3|1.0 |0.1 |-0.6|-1.0|1.3 |-0.3|-0.1|0.3 |-

#### Comparaison avec BLOSUM62
Voici la matrice BLOSUM62
```
  A   R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V   B   Z   X
  4 
 -1   5 
 -2   0   6 
 -2  -2   1   6 
  0  -3  -3  -3   9 
 -1   1   0   0  -3   5 
 -1   0   0   2  -4   2   5 
  0  -2   0  -1  -3  -2  -2   6 
 -2   0   1  -1  -3   0   0  -2   8 
 -1  -3  -3  -3  -1  -3  -3  -4  -3   4 
 -1  -2  -3  -4  -1  -2  -3  -4  -3   2   4 
 -1   2   0  -1  -3   1   1  -2  -1  -3  -2   5 
 -1  -1  -2  -3  -1   0  -2  -3  -2   1   2  -1   5 
 -2  -3  -3  -3  -2  -3  -3  -3  -1   0   0  -3   0   6 
 -1  -2  -2  -1  -3  -1  -1  -2  -2  -3  -3  -1  -2  -4   7 
  1  -1   1   0  -1   0   0   0  -1  -2  -2   0  -1  -2  -1   4 
  0  -1   0  -1  -1  -1  -1  -2  -2  -1  -1  -1  -1  -2  -1   1   5 
 -3  -3  -4  -4  -2  -2  -3  -2  -2  -3  -2  -3  -1   1  -4  -3  -2  11 
 -2  -2  -2  -3  -2  -1  -2  -3   2  -1  -1  -2  -1   3  -3  -2  -2   2   7 
  0  -3  -3  -3  -1  -2  -2  -3  -3   3   1  -2   1  -1  -2  -2   0  -3  -1   4 
 -2  -1   3   4  -3   0   1  -1   0  -3  -4   0  -3  -3  -2   0  -1  -4  -3  -3   4
 -1   0   0   1  -3   3   4  -2   0  -3  -3   1  -1  -3  -1   0  -1  -3  -2  -2   1   4
  0  -1  -1  -1  -2  -1  -1  -1  -1  -1  -1  -1  -1  -1  -2   0   0  -2  -1  -1  -1  -1  -1
```
On peut remarquer que la matrice SH3 40% a des valeurs bien inférieures, tandis que la matrice SH3 70% se rapproche beaucoup plus de la matrice BLOSUM62.         
On remarque cependant quelques grands changements.  Par exemple avec la matrice PDZ 40% où l'on peut voir sur la diagonale des valeurs négatives.  Ce qui signifie que deux lettres identiques seraient plutôt négatives...  Il y a donc bien des substitutions qui ne sont plus acceptées alors qu'elles l'étaient avant.

Ces matrices peuvent être utilisées pour calculer des alignements.  On peut donc comparer le résultats de l'alignement des séquences SH3 avec les différentes matrices de substitution:
```
=-=-=-=-=-= Alignement Global =-=-=-=-=-=
---------- BLOSUM 62 ----------
P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
              .: ::... .. .::::.:. :...:.   . :   .:. :. :.::.::.  .  
P62993: ---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH


----------- SH3 40% -----------
P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
          . . .: ::...... .::::.:. :...:.  .. :   .:. :. :.::.::.  ...
P62993: --MEA-IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH

P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
              .: ::...... .::::.:. :...:.  .. :   .:. :. :.::.::.  ...
P62993: ---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH


----------- SH3 70% -----------
P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
           .. .: ::........::::.:. :...:.. .. : . .:. :..:.::.::.  . .
P62993: ---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH


=-=-=-=-=-= Alignement Local =-=-=-=-=-=
---------- BLOSUM 62 ----------
P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: VALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYV
        .: ::... .. .::::.:. :...:.   . :   .:. :. :.::.::.
P62993: IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYI


----------- SH3 40% -----------
P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: VALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYV
        .: ::...... .::::.:. :...:.  .. :   .:. :. :.::.::.
P62993: IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYI

P12931: VALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
        .: ::...... .::::.:. :...:.  .. :   .:. :. :.::.::.  ...
P62993: IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH


----------- SH3 70% -----------
P12931: GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS (taille: 62)
P62993: MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELNGKDGFIPKNYIEMKPH (taille: 58)

P12931: VALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYV
        .: ::........::::.:. :...:.. .. : . .:. :..:.::.::.
P62993: IAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYI
```
On remarque que la plupart des alignements sont les mêmes, à la seule différence près que les similitudes (entre deux séquences) sont plus importantes au niveau des matrices de substitution créées avec SH3.  Ce qui est logique vu que l'on se base sur les mêmes données pour l'alignement et pour la création de la matrice de substitution.